In [1]:
import numpy as np
import pandas as pd
from src.mcts import mcts
from src.node import MCTSNode
from src.open_ai import get_candidates_fn, get_simulation_fn, get_reward_fn
from src.utils import create_graph_html
import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

np.random.seed(1)

In [2]:
ORIGINAL = "In essence, over a period of time several ships had encountered \"an enormous thing\" at sea, a long spindle-shaped object, sometimes giving off a phosphorescent glow, infinitely bigger and faster than any whale."
GOOGLE_TRANSLATE = "The story goes something like this: Not long ago, a number of large ships encountered a \"monster\" at sea, a very long object, shaped like a spindle, sometimes emitting phosphorescence. It was much larger than a whale and moved much faster than a whale."
CHINESE = "这事大体是这样：不久以前，好些大船在海上碰见了一一个“庞然大物”，一个很长的物体，形状很像纺锤，有时发出磷光，它的体积比鲸鱼大得多，行动起来也比鲸鱼快得多。"

In [3]:
def get_generation_prompt() -> str:
    return f"""Chinese text needs to be translated into English.
- Do not provide any context or description, just the translation.
- A user will start the translation. Complete the translation without repeating what has already been translated.

Translate the following:
{CHINESE}
"""

In [4]:
def get_reward_prompt() -> str:
    return """Provide scores between 0 and 1 of how well the english has been translated from Chinese. Respond in json format for the following keys:
- 'correctness' value between 0 and 1 - if each pinyin token correctly translates into english tokens.
- 'brevity' value between 0 and 1 - if there is no redundancy in the translation.
- 'elegance' value between 0 and 1 - if the translation matches the original prose and is pleasurable to read.

Example:
Pinyin: shuǐ dī shí chuān.
English: Dropping water can penetrate the stone, sometimes.
Response: {example}

Chinese: 學而時習之，不亦悅乎？
English: To learn and to practice what is learned time and again is pleasure, is it not?
Response: {example2}

Translate the following:
{pinyin}""".format(
        pinyin=CHINESE,
        example='{"completeness": 1.0, "correctness": 0.6, "elegance": 0.5}',
        example2='{"completeness": 1.0, "correctness": 0.95, "elegance": 1.0}',
    )

In [5]:
root, node, history = mcts(
    get_candidates_fn=get_candidates_fn(
        prompt=get_generation_prompt(),
        # Consider candidates with at most 10 tokens.
        max_completion_tokens=10,
        # Consider 3 alternatives at each step.
        top_logprobs=3,
        # Consider candidates with at least 5 tokens
        minimum_candidate_token_length=7,
    ),
    get_simulation_fn=get_simulation_fn(prompt=get_generation_prompt()),
    get_rewards_fn=get_reward_fn(prompt=get_reward_prompt()),
    max_rollouts=8,
)

INFO:src.mcts:{'actions': [['root']]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.mcts:{'step': 0, 'actions': [['expansion'], ['simulation'], ['reward']], 'reward': 0.933}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.mcts:{'step': 1, 'actions': [['selection'], ['expansion'], ['simulation'], ['reward']], 'reward': 0.933}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Reque

In [6]:
best_reward_state, best_reward = max(
    [v for i in history for k, v in i.items() if k == "reward"], key=lambda x: x[1]
)

In [7]:
one_shot = get_simulation_fn(prompt=get_generation_prompt())(
    MCTSNode(prob=0.0, state="", type="one_shot")
)

In [8]:
print(
    pd.DataFrame(
        {
            "type": [
                "Chinese",
                "Author's Translation",
                "Google Translate",
                "One Shot",
                "MCTS",
            ],
            "text": [
                CHINESE,
                ORIGINAL,
                GOOGLE_TRANSLATE,
                one_shot,
                best_reward_state,
            ],
        }
    ).to_markdown()
)

|    | type                 | text                                                                                                                                                                                                                                                                                          |
|---:|:---------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Chinese              | 这事大体是这样：不久以前，好些大船在海上碰见了一一个“庞然大物”，一个很长的物体，形状很像纺锤，有时发出磷光，它的体积比鲸鱼大得多，行动起来也比鲸鱼快得多。                                                                                                                                    |
|  1 | Author's Translation | In essence, over a period of time several ships had encountered "an enormous thing" at sea, 

In [9]:
create_graph_html(root=root, filename="graph.html")

graph.html
